In [1]:
import BioSimSpace as BSS
import sys

try:
    import pipeline
except:
    print("adding code to the pythonpath...")
    code = '/home/anna/Documents/code/python'
    if code not in sys.path:
        sys.path.insert(1, code)
    import pipeline

from pipeline import *


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



/home/anna/BioSimSpace/python/BioSimSpace/FreeEnergy/_relative.py:1062: DeprecationWarning: invalid escape sequence \d
  match = _re.search("temp0=([\d.]+)", line)
/home/anna/BioSimSpace/python/BioSimSpace/FreeEnergy/_relative.py:785: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if method is not "alchemlyb":
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.

adding code to the pythonpath...


In [12]:
def my_function(a, b, **kwargs):
    print(a)
    print(b)
    for key,value in kwargs.items():
        if key == "autoequilibration":
            auto_eq = value
        else:
            auto_eq = True

        if key == "statistical_inefficiency":
            stat_ineff = value
        else:
            stat_ineff = False

        if key == "truncate_output":
            truncate = value
        else:
            truncate = 0

In [13]:
my_function(1,2,pipe=True)

1
2
True


In [ ]:



    @staticmethod
    def _truncate_extracted_data(data, truncate, truncate_keep):
         """truncate the extracted output as a percentage


        """       

        if truncate_keep == "start":
            truncated_data = 0 

        return truncated_data

In [9]:

data = BSS.FreeEnergy.Relative._somd_extract_u_nk("/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/bound_0/lambda_0.0000/simfile.dat",300)
# print(type(data))
# print(data)
print(len(data))
print(data.iloc[:, 0])

5000
time    lambdas
0.0     0.0        0.0
0.8     0.0        0.0
1.6     0.0        0.0
2.4     0.0        0.0
3.2     0.0        0.0
                  ... 
3996.0  0.0        0.0
3996.8  0.0        0.0
3997.6  0.0        0.0
3998.4  0.0        0.0
3999.2  0.0        0.0
Name: 0.0, Length: 5000, dtype: float64


In [5]:
def _truncate_extracted_data(data, truncate, truncate_keep):

    perc_kept = int(len(data) * (truncate/100))
    print(perc_kept)

    if truncate_keep == "start":
        truncated_data = data.iloc[:perc_kept, :]
    
    if truncate_keep == "end":
        truncated_data = data.iloc[(len(data) - perc_kept):, :]
    
    print(len(truncated_data))

    return truncated_data

In [6]:
truncated_data = _truncate_extracted_data(data, 25, truncate_keep="end") #BSS.FreeEnergy.Relative.
print(truncated_data)

1250
1250
                0.0       0.1        0.2        0.3        0.4        0.5  \
time   lambdas                                                              
3000.0 0.0      0.0  3.590150   9.488722  17.457728  27.328551  38.971538   
3000.8 0.0      0.0  5.297473  12.747583  22.110646  33.202654  45.845378   
3001.6 0.0      0.0  5.012982  12.240624  21.457811  32.491949  45.175005   
3002.4 0.0      0.0  5.418138  13.010177  22.530506  33.793812  46.626914   
3003.2 0.0      0.0  2.134788   6.534271  12.943868  21.162476  30.983584   
...             ...       ...        ...        ...        ...        ...   
3996.0 0.0      0.0  4.240756  10.783506  19.414135  29.981540  42.381722   
3996.8 0.0      0.0  4.916069  11.975053  20.939853  31.624245  43.838918   
3997.6 0.0      0.0  4.509093  11.237753  19.956575  30.501077  42.754980   
3998.4 0.0      0.0  4.196501  10.615720  19.016323  29.212786  41.025075   
3999.2 0.0      0.0  3.644286   9.557437  17.501458  27.294786  38

In [1]:
import BioSimSpace as BSS


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



/home/anna/BioSimSpace/python/BioSimSpace/FreeEnergy/_relative.py:1165: DeprecationWarning: invalid escape sequence \d
  match = _re.search("temp0=([\d.]+)", line)
/home/anna/BioSimSpace/python/BioSimSpace/FreeEnergy/_relative.py:785: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if method is not "alchemlyb":
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.

In [6]:
process_dict = {"autoequilibration":False, 
                "statistical_inefficiency":False,
                "truncate_output":50,
                "truncate_keep":"end"}

pmf_bound, overlap_matrix_bound = BSS.FreeEnergy.Relative.analyse(
                        f'/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/bound_0',
                        estimator="MBAR",
                        method="alchemlyb",
                        **process_dict
                        )

pmf_free, overlap_matrix_free = BSS.FreeEnergy.Relative.analyse(
                        f'/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/free_0',
                        estimator="MBAR",
                        method="alchemlyb",
                        **process_dict
                        )

freenrg_rel = BSS.FreeEnergy.Relative.difference(pmf_bound, pmf_free)
print(freenrg_rel)

(0.2081 kcal/mol, 0.0648 kcal/mol)


start is (0.2081 kcal/mol, 0.0648 kcal/mol)

In [17]:
from glob import glob
import re as _re

In [21]:

with open("/home/anna/Documents/benchmark/tyk2_benchmark/outputs/AMBER_extracted/lig_ejm31~lig_ejm42/bound_0/lambda_0.0000/amber.out","r") as file:
    for line in file:
        match = _re.search("temp0=([\d.]+)", line)
        if match:
            print(match)


<re.Match object; span=(3, 15), match='temp0=300.00'>


In [13]:
("/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/bound_0/lambda_0.0000/simfile.dat",300)

files = sorted(glob("/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/bound_0/lambda_*/simfile.dat"))
print(files)
u_nk = [BSS.FreeEnergy.Relative._somd_extract_u_nk(x, T=300) for x in files]

['/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/bound_0/lambda_0.0000/simfile.dat', '/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/bound_0/lambda_0.1000/simfile.dat', '/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/bound_0/lambda_0.2000/simfile.dat', '/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/bound_0/lambda_0.3000/simfile.dat', '/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/bound_0/lambda_0.4000/simfile.dat', '/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/bound_0/lambda_0.5000/simfile.dat', '/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/bound_0/lambda_0.6000/simfile.dat', '/home/anna/Documents/benchmark/tyk2_benchmark/outputs/SOMD_extracted/lig_ejm31~lig_ejm42/bound_0/lambda_0.7000/simfi

In [27]:
u_nk[0]

,,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
time,lambdas,,,,,,,,,,,
0.0,0.0,0.0,3.155941,8.486639,15.732104,24.687158,35.145032,46.974545,60.330893,76.557914,102.209412,170.399859
0.8,0.0,0.0,3.407293,9.004699,16.532746,25.788211,36.567279,48.741469,62.460151,78.979032,104.103821,164.604600
1.6,0.0,0.0,3.850045,9.899342,17.911229,27.696764,39.060137,51.852409,66.093025,82.496270,104.771480,151.597985
2.4,0.0,0.0,3.631949,9.495519,17.357033,27.038432,38.383629,51.394511,66.674851,87.372374,128.600520,273.183347
3.2,0.0,0.0,2.580767,7.425930,14.298340,23.010347,33.375576,45.286976,58.923058,75.610983,101.578092,166.922933
...,...,...,...,...,...,...,...,...,...,...,...,...
3996.0,0.0,0.0,4.240756,10.783506,19.414135,29.981540,42.381722,56.765299,74.298015,100.666053,164.841878,448.114729
3996.8,0.0,0.0,4.916069,11.975053,20.939853,31.624245,43.838918,57.458510,72.576892,90.126102,114.281220,162.890059
3997.6,0.0,0.0,4.509093,11.237753,19.956575,30.501077,42.754980,56.870030,74.101430,100.855691,171.906861,526.389775


In [40]:
print(len(u_nk[0]))

df = u_nk[0]
round((u_nk[0].index[-1][0] - u_nk[0].index[-2][0]),1)

5000


0.8

In [49]:
truncate_keep = "start"
truncate = 50

data_len = len(data[0]) # use just the first window for this
data_step = round((data[0].index[-1][0] - data[0].index[-2][0]),1)
data_kept = data_len * (truncate/100)
data_time = data_kept * data_step
print(data_time)
if truncate_keep == "start":
    truncate_lower = 0
    truncate_upper = data_time - data_step
if truncate_keep == "end":
    truncate_lower = (data_len * data_step) - data_time
    truncate_upper = (data_len * data_step) - data_step

print(truncate_lower)
print(truncate_upper)

2000.0
0
1999.2


In [ ]:
    @staticmethod
    def _truncate_extracted_data(data, truncate, truncate_keep):
        """ truncate the data

        Parameters
        ----------

            data : pandas.DataFrame
                Dataframe of extracted dHdl or u_nk data.
            
            truncate: float
                percentage / decimal of how much of the data
                is to be kept.
            
            truncate_keep: str
                "start" or "end". Whether to keep the start or
                the end of the data.

        Returns
        -------

            processed_data : pandas.DataFrame
            Dataframe of dHdl or u_nk data processed to give the specified
            amount of data to be kept
        """
        truncated_data = []

        for df in data:
            print(len(df))
            perc_kept = int(len(df) * (truncate/100))
            print(perc_kept)

            if truncate_keep == "start":
                trunc_data = df.iloc[:perc_kept, :]
            elif truncate_keep == "end":
                trunc_data = df.iloc[(len(df) - perc_kept):, :]
            else:
                raise ValueError("'truncate_keep' must be either 'start' or 'end'.")

            print(len(trunc_data))
            truncated_data.append(trunc_data)

        return _alchemlyb.concat(truncated_data)